## 커스텀 레이어
##### url: https://www.tensorflow.org/tutorials/customization/custom_layers

- 목차
    - 레이어: 유용한 연산들의 일반적 집합
    - 커스텀 레이어 구현하기
    - 모델: 레이어로 구성되는 모델
     
신경망을 만들기 위해 고수준 API로써 `tf.keras`를 사용하기를 추천합니다.<br>
이는, 대부분의 Tensorflow API는 '즉시 실행'(eager execution)으로 사용할 수 있기 때문입니다.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

In [3]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


## 레이어: 유용한 연산들의 일반적인 집합
머신러닝 모델을 작성할 때 대부분의 경우, 독립적인 연산과 독립적인 변수를 다루는 것보다 더 높은 수준의 추상화로 작동하길 원합니다.<br><br>
많은 머신러닝 모델은 비교적 간단한 레이어의 적층이나 구성으로서 표현되어집니다. 또한, Tensorflow는 일반적으로 알려진 많은 레이어 뿐만아니라 처음부터 구현하거나 기존의 레이어의 조합으로써 사용자의 어플리케이션에 특정화된 레이어를 제공합니다. <br><br>
Tensorflow는 tf.keras 패키지에 Keras API의 전체를 포함하고 있으며, 모델을 작성할 때 Keras 레이어는 매우 유용합니다.

In [4]:
# tf.keras.layers 패키지에 들어있는 레이어는 객체입니다. 레이어를 생성하기 위해서는,
# 간단히 객체를 생성하면 됩니다. 대부분의 레이어는 첫 번째 인자로써 출력 차원의 수 / 채널을 입력받습니다.
layer = tf.keras.layers.Dense(100)
# 입력 차원의 수는 종종 필요하지 않을 수 있습니다. 레이어가 처음 사용되어질 때
# 추론될 수 있기 때문입니다. 하지만 수동으로 입력해서 해당 값을 제공할 수 있다면
# 모델의 복잡도에 도움이될 수 있습니다.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

존재하는 레이어의 전체 목록은 [the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers) 에서 확인할 수 있습니다. Dense(a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout 등과 같은 많은 레이어가 있습니다.

In [5]:
# 레이어를 사용하기 위해서는 간단하게 호출하면 됩니다.
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
# 레이어는 많은 유용한 메소드를 가지고 있습니다. 예를 들어, `layer.variable`과 `layer.trainable_variables`를 사용하는 학습 가능한 변수를
# 확인할 수 있습니다, fully-connected layer의 경우 가중치와 편향 값을 가지고 있습니다.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.51255995,  0.56253296, -0.31079015, -0.25119784, -0.5795602 ,
         -0.18677905, -0.08327723, -0.28269935,  0.11898559,  0.37829572],
        [-0.21857187, -0.21728158, -0.13565594,  0.24794936,  0.40024012,
         -0.4996394 , -0.46437404,  0.45892304, -0.37688872,  0.05543399],
        [ 0.34183222, -0.42676014, -0.4025915 , -0.32218546,  0.5161477 ,
         -0.37486407, -0.22887743, -0.04957432,  0.19962084, -0.1280663 ],
        [ 0.06438488,  0.5820032 , -0.40724364, -0.2856705 , -0.27075502,
          0.46156865,  0.08514231,  0.60878223,  0.5578609 ,  0.51370364],
        [ 0.470622  , -0.01868969,  0.3216275 , -0.10308278, -0.00928712,
          0.23166353,  0.25687778,  0.27252483,  0.27809215, -0.4847046 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [7]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.51255995,  0.56253296, -0.31079015, -0.25119784, -0.5795602 ,
         -0.18677905, -0.08327723, -0.28269935,  0.11898559,  0.37829572],
        [-0.21857187, -0.21728158, -0.13565594,  0.24794936,  0.40024012,
         -0.4996394 , -0.46437404,  0.45892304, -0.37688872,  0.05543399],
        [ 0.34183222, -0.42676014, -0.4025915 , -0.32218546,  0.5161477 ,
         -0.37486407, -0.22887743, -0.04957432,  0.19962084, -0.1280663 ],
        [ 0.06438488,  0.5820032 , -0.40724364, -0.2856705 , -0.27075502,
          0.46156865,  0.08514231,  0.60878223,  0.5578609 ,  0.51370364],
        [ 0.470622  , -0.01868969,  0.3216275 , -0.10308278, -0.00928712,
          0.23166353,  0.25687778,  0.27252483,  0.27809215, -0.4847046 ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

## 커스텀 레이어 구현하기
자신만의 레이어를 구현하는 가장 좋은 방법은 tf.keras.Layer 클래스를 확장하고 구현하는 것 입니다.<br>
1. `__init__`, 입력에 독립적으로 초기화할 수 있습니다.
2. `build`, 입력 텐서의 차원을 알게되거나, 초기화를 진행할 수 있습니다.
3. `call`, 순방향으로 연산을 진행합니다. 
변수를 생성하기 위해 `build`가 호출될 때까지 기다릴 수 없다는 것을 명심해야합니다. `__init__` 함수 내에서 생성할 수 있습니다.<br>
하지만, 변수를 `build`내에서 생성하는 것에 대한 이점은 레이어가 작동할 때 들어오는 입력의 차원을 기반으로 변수를 생성할 수 있다는 점 입니다.<br>
다르게 말하면, `__init__`에서 변수를 생성할 경우에는 미리 생성할 변수의 차원을 알고 있어야 한다는 의미입니다.

In [8]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_weight('kernel',
                                      shape=[int(input_shape[-1]),
                                             self.num_outputs])
    
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
layer = MyDenseLayer(10)

In [9]:
_ = layer(tf.zeros([10, 5])) # 레이어를 호출 '.builds' 

In [10]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


가능한 표준 레이어를 사용한다면, 전체 코드는 가독성과 유지보수하기 용이해집니다. 코드를 읽는 다른 사람들은 표준 레이어의 동작 방식에 익숙하기 때문입니다.<br>
만약 `tf.keras.layers`에 존재하지 않는 레이어를 사용한다면, github 이슈를 남기는 것을 고려해보는것이 좋습니다. 풀리퀘스트를 보내주면 더 좋겠네요.

## 모델: 레이어로 구성되는 모델
머신러닝 모델에서 레이어와 같은 많은 흥미로운 것들은 존재하는 레이어의 조합으로 구현되어집니다.<br>
예를 들어, 레즈넷의 각 레지듀얼 블록은 합성곱, 배치 정규화, 숏컷 으로 구성되어 있습니다. 레이어는 다른 레이어들의 내부 중첩으로 구성될 수 있습니다.<br><br>
`Model.fit`, `Model.evaluate` 그리고 `Model.save` 같은 모델 메소드가 필요할 때 일반적으로 `keras.Model`을 상속받게 됩니다. <br><br>
`keras.layers.Layer` 대신에 `keras.Model`를 사용하여 제공받는 특징으로는 변수를 추적할 수 있게됩니다. `keras.Model`은 내부 레이어를 추적할 수 있으며, 검사하기 쉽게 해줍니다. <br><br>
레즈넷 블록의 예를 보겠습니다.

In [11]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a   = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b   = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c   = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        return tf.nn.relu(x)
    
block = ResnetIdentityBlock(1, [1, 2, 3])

In [12]:
_ = block(tf.zeros([1, 2, 3, 3]))

In [13]:
block.layers

In [14]:
len(block.variables)

18

In [15]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

In [16]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                     input_shape=(
                                                         None, None, 3)),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(2, 1,
                                                     padding='same'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(3, (1, 1)),
                              tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [17]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_3 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_4 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_5 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________